In [16]:
import akshare as ak
import pandas as pd
import numpy as np
from tqdm import tqdm

In [4]:
target_index = "399300"

In [6]:
index_stock_info_df = ak.index_stock_info()

In [7]:
index_stock_cons_df = ak.index_stock_cons(symbol=target_index)
print(index_stock_cons_df)

       品种代码  品种名称        纳入日期
0    002756  永兴材料  2022-12-12
1    688303  大全能源  2022-12-12
2    600884  杉杉股份  2022-12-12
3    601689  拓普集团  2022-12-12
4    000723  美锦能源  2022-12-12
..      ...   ...         ...
295  600660  福耀玻璃  2005-04-08
296  600690  青岛海尔  2005-04-08
297  600741  巴士股份  2005-04-08
298  600795  国电电力  2005-04-08
299  600900  长江电力  2005-04-08

[300 rows x 3 columns]


In [9]:
stk_in_name = list(index_stock_cons_df["品种代码"])
print(len(stk_in_name))
print(type(stk_in_name[0]))

300
<class 'str'>


In [17]:
data_ls = []
for stk_name in tqdm(stk_in_name):
    df = ak.stock_zh_a_hist(stk_name, period="daily", start_date="20150101", adjust="qfq")
    # df = ak.stock_us_hist(stk_name, period="daily", start_date="20150101", adjust="hfq")
    df["stk_id"] = stk_name[4:]
    df1 = df.rename(columns={
        "stk_id": "stk_id",
        "日期":"trade_date",
        "开盘":"open",
        "收盘":"close",
        "最高":"high",
        "最低":"low",
        "成交量":"volume",
        "成交额":"amount",
        "振幅":"amplitude",
        "涨跌幅":"pct_change",
        "涨跌额":"change",
        "换手率":"turnover",
    })
    data_ls.append(df1)

100%|██████████| 300/300 [01:23<00:00,  3.57it/s]


In [23]:
df2 = pd.concat(data_ls)
print(df2.shape)
df2.to_csv("hs300_raw_data.csv", index=False)

(511602, 12)


In [24]:
df2 = pd.read_csv("hs300_raw_data.csv")

In [25]:
print(df2.columns)

Index(['trade_date', 'open', 'close', 'high', 'low', 'volume', 'amount',
       'amplitude', 'pct_change', 'change', 'turnover', 'stk_id'],
      dtype='object')


In [1]:
df3 = df2.pivot(index='trade_date', columns='stk_id',
                             values=['open', 'close', 'high', 'low', 'volume', 'amount', 'amplitude', 'pct_change', 'change', 'turnover'])
print(df3)

NameError: name 'df2' is not defined

In [ ]:
amount = df3["amount"].replace(0,np.nan)
data1 = {
    "open": df3["open"].replace(0,np.nan),
    "close": df3["close"].replace(0,np.nan),
    "high": df3["high"].replace(0,np.nan),
    "low": df3["low"].replace(0,np.nan),
    "volume": df3["volume"].replace(0,np.nan),
    #
    # "turnover": df3["turnover"].replace(0,np.nan),
}
data1["adv20"] = data1["volume"].rolling(window=20).mean()
data1["vwap"] = amount / data1["volume"]
data1["returns"] = (data1["close"] / data1["close"].shift(1) - 1)
y = (data1["close"] / data1["close"].shift(1) - 1)

In [ ]:
start_date = "2017-07-11"
end_date = "2023-04-13"
# val[(pd.Series(val.index) < start_date) | (pd.Series(val.index) > end_date)].index
for key, val in data1.items():
    data1[key] = val.loc[start_date:end_date,:]
y = y.loc[start_date:end_date,:]

In [ ]:
# 删除缺失值占比>0.1的股票，并填充缺失值
data2 = data1.copy()
y1 = y.copy()
index_list = ['open', 'close', 'high', 'low', 'volume', 'adv20', 'vwap', 'returns']
drop_df = pd.DataFrame(columns=next(iter(data2.values())).columns, index=index_list)
threshold = next(iter(data2.values())).shape[0] * 0.1
print(drop_df.shape)
print(data2["open"].shape)
print(y.shape)
for idx, df in data2.items():
    drop_df.loc[idx] = df.isna().sum() > threshold
mask = drop_df.sum() > 1
for idx, df in data2.items():
    data2[idx] = data2[idx].loc[:, ~mask.values]
    data2[idx].fillna(method='ffill',inplace=True)
    data2[idx].fillna(method='bfill',inplace=True)
y = y.loc[:, ~mask.values]
y.fillna(method='ffill',inplace=True)
y.fillna(method='bfill',inplace=True)

In [ ]:
# 对其y和data


In [ ]:
import pickle
with open("us_data_2017_2023.pkl", "wb") as f:
    pickle.dump(data2, f)
with open("us_return_2017_2023.pkl", "wb") as f:
    pickle.dump(y2, f)